<a href="https://colab.research.google.com/github/ZahraDehghani99/AI-Internship/blob/master/Speech%20models/Speech_Emotion_Recognition_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speech Emotion Recognition

https://huggingface.co/docs/transformers/tasks/audio_classification#audio-classification

In [1]:
!pip install -q transformers datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00

In [2]:
!pip install -q --upgrade datasets transformers

In [3]:
!pip install -q accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda')

In [7]:
!nvidia-smi

Fri Jul 21 08:20:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    42W / 400W |      3MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import wandb

wandb.login(key='c25b22f09be7fa6fccb297ac2f752ec4ebf70a98')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Load dataset from hugging face hub

In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
from datasets import load_dataset, Audio

iemocap_audio = load_dataset("Zahra99/IEMOCAP_Audio")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating session1 split:   0%|          | 0/1085 [00:00<?, ? examples/s]

Generating session2 split:   0%|          | 0/1023 [00:00<?, ? examples/s]

Generating session3 split:   0%|          | 0/1151 [00:00<?, ? examples/s]

Generating session4 split:   0%|          | 0/1031 [00:00<?, ? examples/s]

Generating session5 split:   0%|          | 0/1241 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Zahra99___parquet/Zahra99--IEMOCAP_Audio-26a2da2dd22053c2/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
iemocap_audio

DatasetDict({
    session1: Dataset({
        features: ['audio', 'label'],
        num_rows: 1085
    })
    session2: Dataset({
        features: ['audio', 'label'],
        num_rows: 1023
    })
    session3: Dataset({
        features: ['audio', 'label'],
        num_rows: 1151
    })
    session4: Dataset({
        features: ['audio', 'label'],
        num_rows: 1031
    })
    session5: Dataset({
        features: ['audio', 'label'],
        num_rows: 1241
    })
})

In [12]:
iemocap_audio["session1"].features

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'label': ClassLabel(names=['ang', 'hap', 'neu', 'sad'], id=None)}

for path we have only the file name not all of the directory, although we have full directory before we create hugging face dataset.

In [13]:
iemocap_audio["session1"][0]

{'audio': {'path': 'Ses01M_impro01_F000.wav',
  'array': array([ 0.00543213,  0.00488281,  0.00537109, ...,  0.0017395 ,
         -0.00650024, -0.01904297]),
  'sampling_rate': 16000},
 'label': 0}

In [14]:
labels = iemocap_audio["session1"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [15]:
id2label[str(0)]

'ang'

## Define needed functions

### Defining the performance metrics

In [16]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

### Defining Confusion matrix

In [17]:
classes = ["ang", "hap", "neu", "sad"]

In [18]:
def confusion_matrix(y_test, y_pred):
  labels = [i for i in range(len(list(set(y_test))))]
  df_cm = pd.DataFrame(metrics.confusion_matrix(y_test, y_pred), index = labels,
                columns = labels)
  plt.figure(figsize=(8,6))
  ax= plt.subplot()
  sns.heatmap(df_cm, annot=True, cmap="YlGnBu" ,fmt='g', ax=ax)
  plt.title('Confusion matrix', y=1.1)
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')
  class_names= classes
  ax.xaxis.set_ticklabels(class_names); ax.yaxis.set_ticklabels(class_names);
  plt.tight_layout()
  plt.show()

In [ ]:
iemocap_audio["session1"][0]

{'audio': {'path': 'Ses01M_impro01_F000.wav',
  'array': array([ 0.00543213,  0.00488281,  0.00537109, ...,  0.0017395 ,
         -0.00650024, -0.01904297]),
  'sampling_rate': 16000},
 'label': 0}

In [ ]:
labels = iemocap_audio["session1"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
id2label[str(0)]

'ang'

## Leave One Session Out (Wav2vec)

In [19]:
from datasets import concatenate_datasets
from datasets import DatasetDict

from transformers import set_seed
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForAudioClassification

set_seed(42)

In [20]:
from transformers import AutoFeatureExtractor

model_checkpoint = "facebook/wav2vec2-base"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

In [21]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [22]:
num_labels = 4
def model_init():
    return AutoModelForAudioClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [23]:
dataset = iemocap_audio

In [24]:
cv_scores = {'Accuracy' : [], 'F1' : []}
for i in range(1, 6):
    test_split = [f"session{i}"]
    train_splits = [f"session{i}" for i in range(1, 6)]
    train_splits.remove(f"session{i}")


    iemocap_train = concatenate_datasets([dataset[i] for i in train_splits])
    iemocap_test = dataset[test_split[0]]

    shuffled_iemocap_train = iemocap_train.shuffle(seed=42)

    fold_dataset = DatasetDict({
                "train": shuffled_iemocap_train,
                "test": iemocap_test})

    fold_dataset_encoded = fold_dataset.map(preprocess_function, remove_columns="audio", batched=True)


    batch_size = 32
    # logging_steps = len(fold_dataset_encoded["train"]) // batch_size
    model_name = f"{model_checkpoint}-finetuned-iemocap{i}-LOSO"

    # Initialize WandB
    wandb.init(name= model_name)

    training_args = TrainingArguments(output_dir=model_name,
                                      num_train_epochs=20,
                                      learning_rate=3e-5,
                                      seed=42,
                                      per_device_train_batch_size=batch_size,
                                      per_device_eval_batch_size=batch_size,
                                      evaluation_strategy="epoch",
                                      save_strategy="epoch",
                                      warmup_ratio=0.1, ###?
                                      logging_steps=10,
                                      load_best_model_at_end=True,
                                      lr_scheduler_type='polynomial',
                                      disable_tqdm=False,
                                      metric_for_best_model="accuracy",
                                      push_to_hub=True,
                                      hub_private_repo=True,
                                      log_level="error",
                                      report_to = 'wandb')

    trainer = Trainer(model_init=model_init,
                      args=training_args,
                      compute_metrics=compute_metrics,
                      train_dataset=fold_dataset_encoded["train"],
                      eval_dataset=fold_dataset_encoded["test"],
                      tokenizer=feature_extractor)

    torch.cuda.empty_cache()

    trainer.train()

    trainer.push_to_hub()

    preds_output_test = trainer.predict(fold_dataset_encoded["test"])
    acc = preds_output_test.metrics["test_accuracy"]
    f1 = preds_output_test.metrics["test_f1"]

    cv_scores['Accuracy'].append(acc)
    cv_scores['F1'].append(f1)

mean_cv_scores = {k: np.mean(v) for k, v in cv_scores.items()}
print(f"mean accuracy: {mean_cv_scores['Accuracy']:0.2f}\nmean f1-score: {mean_cv_scores['F1']:0.2f}")

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

wandb: Currently logged in as: zdehghanitafti78 (affective-computing). Use `wandb login --relogin` to force relogin


Cloning https://huggingface.co/Zahra99/wav2vec2-base-finetuned-iemocap1-LOSO into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.40k/361M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.248000,1.220047,0.456890,0.450124
2,1.161800,1.148524,0.469782,0.436086
3,1.127200,1.098375,0.505238,0.497069
4,1.045800,1.131792,0.493151,0.487103
5,1.041300,1.145828,0.488316,0.474600
6,0.922500,1.105385,0.515713,0.492788
7,0.882100,1.102023,0.519742,0.505626
8,0.874000,1.083286,0.538276,0.521329
9,0.755800,1.090660,0.556809,0.551706
10,0.756800,1.122294,0.544722,0.531444


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/361M [00:00<?, ?B/s]

To https://huggingface.co/Zahra99/wav2vec2-base-finetuned-iemocap5-LOSO
   c0bb193..9655667  main -> main

   c0bb193..9655667  main -> main

To https://huggingface.co/Zahra99/wav2vec2-base-finetuned-iemocap5-LOSO
   9655667..e1cf5c6  main -> main

   9655667..e1cf5c6  main -> main



mean accuracy: 0.57
mean f1-score: 0.56


In [ ]:
fold_dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4446
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1085
    })
})